<a href="https://colab.research.google.com/github/baysalmf/Diabetes-Prediction/blob/main/LSTMGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install workalendar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import itertools
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from workalendar.europe import Turkey

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.0 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732003 sha256=60794e30f2a3e6ce2449c4b87290d4f54646cb16df71710ce5075b37a2b2ca37
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built pymeeus


In [ ]:
consumption = pd.read_excel(r'Dagıtılan.xlsx')
meteo = pd.read_excel(r'Hava.xlsx')

In [ ]:
df = pd.merge(consumption, meteo, on='Tarih', how='inner')

In [ ]:
df['Tarih'] = pd.to_datetime(df['Tarih']).dt.normalize()

In [ ]:
df.head()

,Tarih,Saat,Dağıtılan,Sıcaklık,Nem
0,2019-12-31,0,557.057514,3.1,96.0
1,2020-01-01,1,510.814986,3.4,NaN
2,2020-01-01,2,478.842714,3.3,100.0
3,2020-01-01,3,459.028750,3.3,NaN
4,2020-01-01,4,450.351698,3.6,100.0


In [ ]:
def create_date_features(dataframe):
    dataframe['Yıl'] = dataframe['Tarih'].dt.year
    dataframe['Ay'] = dataframe['Tarih'].dt.month
    dataframe['Gün'] = dataframe['Tarih'].dt.day
    dataframe['Haftanıngünleri'] = dataframe['Tarih'].dt.dayofweek
    dataframe['Mevsim'] = dataframe['Tarih'].dt.quarter
    dataframe['HaftanınGünü'] = dataframe['Tarih'].dt.day_name()
    dataframe['Yılıngünü'] = dataframe['Tarih'].dt.dayofyear
    return dataframe

create_date_features(df)

,Tarih,Saat,Dağıtılan,Sıcaklık,Nem,Yıl,Ay,Gün,Haftanıngünleri,Mevsim,HaftanınGünü,Yılıngünü
0,2019-12-31,0,557.057514,3.1,96.0,2019,12,31,1,4,Tuesday,365
1,2020-01-01,1,510.814986,3.4,NaN,2020,1,1,2,1,Wednesday,1
2,2020-01-01,2,478.842714,3.3,100.0,2020,1,1,2,1,Wednesday,1
3,2020-01-01,3,459.028750,3.3,NaN,2020,1,1,2,1,Wednesday,1
4,2020-01-01,4,450.351698,3.6,100.0,2020,1,1,2,1,Wednesday,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26035,2022-12-20,19,759.240000,5.1,95.0,2022,12,20,1,4,Tuesday,354
26036,2022-12-20,20,733.720000,5.2,95.0,2022,12,20,1,4,Tuesday,354
26037,2022-12-20,21,705.500000,5.1,94.0,2022,12,20,1,4,Tuesday,354
26038,2022-12-20,22,674.120000,5.2,95.0,2022,12,20,1,4,Tuesday,354


In [ ]:
from workalendar.europe import Turkey
cal = Turkey()
all_holidays = []

for year in range(2019, 2022):
    holidays = cal.holidays(year)
    for date, name in holidays:
        all_holidays.append(date.isoformat())

all_holidays = pd.to_datetime(all_holidays)

df['Resmi_tatil_günleri'] = np.where((df['Tarih'].isin(all_holidays)),0, 1)
df.drop('HaftanınGünü', axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/workalendar/core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


In [ ]:
df.head(10)

,Tarih,Saat,Dağıtılan,Sıcaklık,Nem,Yıl,Ay,Gün,Haftanıngünleri,Mevsim,Yılıngünü,Resmi_tatil_günleri
0,2019-12-31,0,557.057514,3.1,96.0,2019,12,31,1,4,365,1
1,2020-01-01,1,510.814986,3.4,NaN,2020,1,1,2,1,1,0
2,2020-01-01,2,478.842714,3.3,100.0,2020,1,1,2,1,1,0
3,2020-01-01,3,459.028750,3.3,NaN,2020,1,1,2,1,1,0
4,2020-01-01,4,450.351698,3.6,100.0,2020,1,1,2,1,1,0
5,2020-01-01,5,450.621970,4.1,97.0,2020,1,1,2,1,1,0
6,2020-01-01,6,463.890206,5.5,89.0,2020,1,1,2,1,1,0
7,2020-01-01,7,465.779606,5.8,82.0,2020,1,1,2,1,1,0
8,2020-01-01,8,479.002762,6.9,85.0,2020,1,1,2,1,1,0
9,2020-01-01,9,542.739254,8.0,63.0,2020,1,1,2,1,1,0


In [ ]:
df.isnull().sum()

,0
Tarih,0
Saat,0
Dağıtılan,0
Sıcaklık,121
Nem,1911
Yıl,0
Ay,0
Gün,0
Haftanıngünleri,0
Mevsim,0


In [ ]:
df['Sıcaklık'].fillna(method='ffill', inplace=True)
df['Nem'].fillna(method='ffill', inplace=True)

<ipython-input-10-86470dfa0bdb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Sıcaklık'].fillna(method='ffill', inplace=True)
<ipython-input-10-86470dfa0bdb>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sıcaklık'].fillna(method='ffill', inplace=True)
<ipython-input-10-86470dfa0bdb>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior

In [ ]:
def random_noise(dataframe):
  return np.random.normal(scale=1.6, size=(len(dataframe),))

In [ ]:
def lag_features(dataframe, lags):
  for lag in lags:
    dataframe['Dağıtılan_lag'+str(lag)] = dataframe['Dağıtılan'].transform(lambda x: x.shift(lag)) + random_noise(dataframe)
  return dataframe

In [ ]:
df = lag_features(df, [24, 48, 96, 120, 144, 168, 192, 216, 240])

In [ ]:
def roll_mean_features(dataframe, windows):
  for window in windows:
    dataframe['dağıtılan'+str(window)] = dataframe['Dağıtılan'].transform(lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type='triang').mean()) + random_noise(dataframe)
  return dataframe

In [ ]:
df = roll_mean_features(df, [8760, 13140])

In [ ]:
def ewm_features(dataframe, alphas, lags):
  for alpha in alphas:
    for lag in lags:
      dataframe['dağıtılan_ewm_alpha_'+str(alpha).replace(".", "")+"_lag_"+str(lag)] = dataframe['Dağıtılan'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
  return dataframe

In [ ]:
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [24, 48, 52, 96, 120, 144, 168, 192, 216]

In [ ]:
df = ewm_features(df, alphas, lags)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x:'%.3f'%x)

In [ ]:
df.tail(15)

,Tarih,Saat,Dağıtılan,Sıcaklık,Nem,Yıl,Ay,Gün,Haftanıngünleri,Mevsim,Yılıngünü,Resmi_tatil_günleri,Dağıtılan_lag24,Dağıtılan_lag48,Dağıtılan_lag96,Dağıtılan_lag120,Dağıtılan_lag144,Dağıtılan_lag168,Dağıtılan_lag192,Dağıtılan_lag216,Dağıtılan_lag240,dağıtılan8760,dağıtılan13140,dağıtılan_ewm_alpha_095_lag_24,dağıtılan_ewm_alpha_095_lag_48,dağıtılan_ewm_alpha_095_lag_52,dağıtılan_ewm_alpha_095_lag_96,dağıtılan_ewm_alpha_095_lag_120,dağıtılan_ewm_alpha_095_lag_144,dağıtılan_ewm_alpha_095_lag_168,dağıtılan_ewm_alpha_095_lag_192,dağıtılan_ewm_alpha_095_lag_216,dağıtılan_ewm_alpha_09_lag_24,dağıtılan_ewm_alpha_09_lag_48,dağıtılan_ewm_alpha_09_lag_52,dağıtılan_ewm_alpha_09_lag_96,dağıtılan_ewm_alpha_09_lag_120,dağıtılan_ewm_alpha_09_lag_144,dağıtılan_ewm_alpha_09_lag_168,dağıtılan_ewm_alpha_09_lag_192,dağıtılan_ewm_alpha_09_lag_216,dağıtılan_ewm_alpha_08_lag_24,dağıtılan_ewm_alpha_08_lag_48,dağıtılan_ewm_alpha_08_lag_52,dağıtılan_ewm_alpha_08_lag_96,dağıtılan_ewm_alpha_08_lag_120,dağıtılan_ewm_alpha_08_lag_144,dağıtılan_ewm_alpha_08_lag_168,dağıtılan_ewm_alpha_08_lag_192,dağıtılan_ewm_alpha_08_lag_216,dağıtılan_ewm_alpha_07_lag_24,dağıtılan_ewm_alpha_07_lag_48,dağıtılan_ewm_alpha_07_lag_52,dağıtılan_ewm_alpha_07_lag_96,dağıtılan_ewm_alpha_07_lag_120,dağıtılan_ewm_alpha_07_lag_144,dağıtılan_ewm_alpha_07_lag_168,dağıtılan_ewm_alpha_07_lag_192,dağıtılan_ewm_alpha_07_lag_216,dağıtılan_ewm_alpha_05_lag_24,dağıtılan_ewm_alpha_05_lag_48,dağıtılan_ewm_alpha_05_lag_52,dağıtılan_ewm_alpha_05_lag_96,dağıtılan_ewm_alpha_05_lag_120,dağıtılan_ewm_alpha_05_lag_144,dağıtılan_ewm_alpha_05_lag_168,dağıtılan_ewm_alpha_05_lag_192,dağıtılan_ewm_alpha_05_lag_216
26025,2022-12-20,9,708.680,5.500,91.000,2022,12,20,1,4,354,1,699.375,548.504,676.464,679.430,695.916,690.963,686.769,547.700,640.079,657.794,656.078,695.599,547.950,488.129,673.691,676.991,693.045,686.687,685.302,547.640,692.321,545.860,488.058,670.199,673.706,689.379,683.061,681.552,545.825,684.483,541.458,488.111,661.961,665.981,680.794,674.505,672.733,541.769,674.623,536.765,488.668,651.817,656.411,670.254,663.956,661.861,537.186,647.211,526.697,494.192,624.570,630.167,641.766,635.529,632.328,526.793
26026,2022-12-20,10,745.500,5.700,93.000,2022,12,20,1,4,354,1,738.113,605.032,718.894,717.534,714.784,726.548,724.987,613.460,680.186,655.997,656.625,736.165,603.135,503.510,715.785,713.499,715.783,723.407,725.276,608.079,733.702,600.022,502.694,713.220,711.249,714.220,721.112,722.797,604.717,727.537,593.124,501.078,706.792,705.532,709.743,715.173,716.451,597.362,719.197,585.257,499.624,698.145,697.717,702.962,706.925,707.724,589.038,692.755,566.369,499.256,671.285,672.794,679.373,680.434,679.854,569.026
26027,2022-12-20,11,763.600,5.400,94.000,2022,12,20,1,4,354,1,760.926,642.081,731.122,729.634,733.509,745.823,743.931,654.382,708.450,657.685,654.830,758.238,640.608,493.792,729.156,727.921,734.115,743.673,741.829,651.685,756.830,638.324,494.221,728.196,726.937,732.994,742.377,740.710,649.054,753.027,632.689,494.840,725.246,724.050,730.013,738.827,737.450,642.656,747.339,625.383,495.183,720.346,719.391,725.445,733.395,732.207,634.497,726.078,604.474,496.268,700.573,700.737,707.226,712.587,711.277,611.503
26028,2022-12-20,12,750.440,5.000,92.000,2022,12,20,1,4,354,1,729.229,654.452,686.333,704.059,700.529,734.239,721.430,655.238,683.496,658.318,658.237,728.942,652.551,509.760,688.177,705.519,703.929,734.617,723.238,653.523,730.343,651.694,508.962,690.238,706.600,705.405,734.964,724.105,653.163,732.525,649.082,507.448,693.865,708.282,707.875,735.077,725.298,651.427,733.382,644.841,505.975,696.318,708.855,709.271,733.917,725.244,647.883,726.739,628.827,503.434,693.296,702.538,704.783,723.364,716.769,632.562
26029,2022-12-20,13,753.160,5.100,92.000,2022,12,20,1,4,354,1,739.875,656.653,692.060,690.947,723.128,744.884,731.306,667.786,678.533,657.284,656.168,740.055,655.239,547.950,691.372,692.638,721.419,742.258,733.861,664.426,739.610,655.011,545.860,691.410,693.424,720.647,741.890,733.388,663.816,739.0

In [ ]:
df['sin_hour'] = np.sin(2*np.pi*df['Saat']/24)
df['cos_hour'] = np.cos(2*np.pi*df['Saat']/24)

In [ ]:
df['sin_month'] = np.sin(2*np.pi*df['Ay']/12)
df['cos_month'] = np.cos(2*np.pi*df['Ay']/12)

In [ ]:
df['sin_week'] = np.sin(2*np.pi*df['Haftanıngünleri']/7)
df['cos_week'] = np.cos(2*np.pi*df['Haftanıngünleri']/7)

In [ ]:
df['sin_day_of_year'] = np.sin(2 * np.pi * df['Yılıngünü'] / 365)
df['cos_day_of_year'] = np.cos(2 * np.pi * df['Yılıngünü'] / 365)

KeyError: 'Yılıngünü'

In [ ]:
df['sin_day_of_month'] = np.sin(2 * np.pi * df['Gün'] / 30)
df['cos_day_of_month'] = np.cos(2 * np.pi * df['Gün'] / 30)

In [ ]:
df.tail(10)

,Tarih,Dağıtılan,Sıcaklık,Nem,Gün,Mevsim,Resmi_tatil_günleri,Dağıtılan_lag24,Dağıtılan_lag48,Dağıtılan_lag96,Dağıtılan_lag120,Dağıtılan_lag144,Dağıtılan_lag168,Dağıtılan_lag192,Dağıtılan_lag216,Dağıtılan_lag240,dağıtılan8760,dağıtılan13140,dağıtılan_ewm_alpha_095_lag_24,dağıtılan_ewm_alpha_095_lag_48,dağıtılan_ewm_alpha_095_lag_52,dağıtılan_ewm_alpha_095_lag_96,dağıtılan_ewm_alpha_095_lag_120,dağıtılan_ewm_alpha_095_lag_144,dağıtılan_ewm_alpha_095_lag_168,dağıtılan_ewm_alpha_095_lag_192,dağıtılan_ewm_alpha_095_lag_216,dağıtılan_ewm_alpha_09_lag_24,dağıtılan_ewm_alpha_09_lag_48,dağıtılan_ewm_alpha_09_lag_52,dağıtılan_ewm_alpha_09_lag_96,dağıtılan_ewm_alpha_09_lag_120,dağıtılan_ewm_alpha_09_lag_144,dağıtılan_ewm_alpha_09_lag_168,dağıtılan_ewm_alpha_09_lag_192,dağıtılan_ewm_alpha_09_lag_216,dağıtılan_ewm_alpha_08_lag_24,dağıtılan_ewm_alpha_08_lag_48,dağıtılan_ewm_alpha_08_lag_52,dağıtılan_ewm_alpha_08_lag_96,dağıtılan_ewm_alpha_08_lag_120,dağıtılan_ewm_alpha_08_lag_144,dağıtılan_ewm_alpha_08_lag_168,dağıtılan_ewm_alpha_08_lag_192,dağıtılan_ewm_alpha_08_lag_216,dağıtılan_ewm_alpha_07_lag_24,dağıtılan_ewm_alpha_07_lag_48,dağıtılan_ewm_alpha_07_lag_52,dağıtılan_ewm_alpha_07_lag_96,dağıtılan_ewm_alpha_07_lag_120,dağıtılan_ewm_alpha_07_lag_144,dağıtılan_ewm_alpha_07_lag_168,dağıtılan_ewm_alpha_07_lag_192,dağıtılan_ewm_alpha_07_lag_216,dağıtılan_ewm_alpha_05_lag_24,dağıtılan_ewm_alpha_05_lag_48,dağıtılan_ewm_alpha_05_lag_52,dağıtılan_ewm_alpha_05_lag_96,dağıtılan_ewm_alpha_05_lag_120,dağıtılan_ewm_alpha_05_lag_144,dağıtılan_ewm_alpha_05_lag_168,dağıtılan_ewm_alpha_05_lag_192,dağıtılan_ewm_alpha_05_lag_216,sin_hour,cos_hour,sin_month,cos_month,sin_week,cos_week,sin_day_of_year,cos_day_of_year,sin_day_of_month,cos_day_of_month
26030,2022-12-20,752.760,5.100,94.000,20,4,1,732.160,644.632,696.566,720.706,731.005,748.528,734.839,659.614,669.559,656.105,658.190,736.735,644.201,603.135,694.230,719.449,728.640,748.017,735.361,660.050,736.865,644.759,600.022,694.083,718.116,728.183,747.677,735.235,660.220,737.051,645.720,593.124,693.905,715.733,727.105,746.885,734.871,660.313,737.131,646.199,585.257,693.958,713.711,725.840,745.835,734.308,659.833,735.125,642.862,566.369,693.399,709.055,721.291,740.666,730.517,654.300,-0.500,-0.866,-0.000,1.000,0.782,0.623,-0.188,0.982,-0.866,-0.500
26031,2022-12-20,747.300,4.900,96.000,20,4,1,734.182,628.327,683.080,714.883,726.690,747.900,732.508,660.696,668.138,659.545,657.983,734.042,627.670,640.608,684.055,714.291,725.676,747.450,731.693,660.193,734.197,628.596,638.324,684.576,714.430,725.786,747.446,731.873,660.202,734.530,630.584,632.689,685.597,714.363,725.837,747.313,732.174,660.223,734.869,632.620,625.383,686.651,713.927,725.616,746.945,732.342,660.090,734.512,634.831,604.474,688.460,711.537,723.405,744.043,731.009,657.250,-0.707,-0.707,-0.000,1.000,0.782,0.623,-0.188,0.982,-0.866,-0.500
26032,2022-12-20,764.300,5.000,95.000,20,4,1,750.266,660.497,711.233,736.213,746.435,763.786,747.734,685.355,687.867,658.510,657.608,752.071,659.429,652.551,711.515,735.295,746.713,762.868,747.337,684.976,751.138,657.850,651.694,710.122,734.203,745.617,762.057,746.531,683.672,749.322,654.997,649.082,707.487,731.993,743.423,760.407,744.963,681.069,747.575,652.556,644.841,705.067,729.658,741.159,758.659,743.415,678.423,743.766,647.965,628.827,700.710,723.969,735.613,753.861,739.584,671.765,-0.866,-0.500,-0.000,1.000,0.782,0.623,-0.188,0.982,-0.866,-0.500
26033,2022-12-20,819.900,5.100,95.000,20,4,1,809.664,774.737,780.566,795.155,794.642,814.194,799.940,772.678,783.764,655.486,658.692,807.066,768.081,655.239,776.082,793.307,792.149,811.728,798.279,767.117,804.078,762.205,655.011,772.544,790.144,789.648,809.076,795.517,762.663,797.832,750.039,654.120,765.081,783.487,784.317,803.521,789.761,753.366,791.244,737.427,652.218,757.156,776.349,778.526,797.608,783.696,743.535,776.863,710.883,642.104,740.095,760.164,765.076,784.081,770.272,721.603,-0.966,-0.259,-0.000,1.000,0.782,0.623,-0.188,0.982,-0.866,-0.500
26034,2022-12-20,7

In [ ]:
df.drop(['Yıl', 'Saat', 'Ay', 'Yılıngünü', 'Haftanıngünleri', 'Gün'], axis=1, inplace=True)

In [ ]:
df.tail(5)

,Tarih,Dağıtılan,Sıcaklık,Nem,Gün,Mevsim,Resmi_tatil_günleri,Dağıtılan_lag24,Dağıtılan_lag48,Dağıtılan_lag96,Dağıtılan_lag120,Dağıtılan_lag144,Dağıtılan_lag168,Dağıtılan_lag192,Dağıtılan_lag216,Dağıtılan_lag240,dağıtılan8760,dağıtılan13140,dağıtılan_ewm_alpha_095_lag_24,dağıtılan_ewm_alpha_095_lag_48,dağıtılan_ewm_alpha_095_lag_52,dağıtılan_ewm_alpha_095_lag_96,dağıtılan_ewm_alpha_095_lag_120,dağıtılan_ewm_alpha_095_lag_144,dağıtılan_ewm_alpha_095_lag_168,dağıtılan_ewm_alpha_095_lag_192,dağıtılan_ewm_alpha_095_lag_216,dağıtılan_ewm_alpha_09_lag_24,dağıtılan_ewm_alpha_09_lag_48,dağıtılan_ewm_alpha_09_lag_52,dağıtılan_ewm_alpha_09_lag_96,dağıtılan_ewm_alpha_09_lag_120,dağıtılan_ewm_alpha_09_lag_144,dağıtılan_ewm_alpha_09_lag_168,dağıtılan_ewm_alpha_09_lag_192,dağıtılan_ewm_alpha_09_lag_216,dağıtılan_ewm_alpha_08_lag_24,dağıtılan_ewm_alpha_08_lag_48,dağıtılan_ewm_alpha_08_lag_52,dağıtılan_ewm_alpha_08_lag_96,dağıtılan_ewm_alpha_08_lag_120,dağıtılan_ewm_alpha_08_lag_144,dağıtılan_ewm_alpha_08_lag_168,dağıtılan_ewm_alpha_08_lag_192,dağıtılan_ewm_alpha_08_lag_216,dağıtılan_ewm_alpha_07_lag_24,dağıtılan_ewm_alpha_07_lag_48,dağıtılan_ewm_alpha_07_lag_52,dağıtılan_ewm_alpha_07_lag_96,dağıtılan_ewm_alpha_07_lag_120,dağıtılan_ewm_alpha_07_lag_144,dağıtılan_ewm_alpha_07_lag_168,dağıtılan_ewm_alpha_07_lag_192,dağıtılan_ewm_alpha_07_lag_216,dağıtılan_ewm_alpha_05_lag_24,dağıtılan_ewm_alpha_05_lag_48,dağıtılan_ewm_alpha_05_lag_52,dağıtılan_ewm_alpha_05_lag_96,dağıtılan_ewm_alpha_05_lag_120,dağıtılan_ewm_alpha_05_lag_144,dağıtılan_ewm_alpha_05_lag_168,dağıtılan_ewm_alpha_05_lag_192,dağıtılan_ewm_alpha_05_lag_216,sin_hour,cos_hour,sin_month,cos_month,sin_week,cos_week,sin_day_of_year,cos_day_of_year
26035,2022-12-20,759.240,5.100,95.000,20,4,1,753.715,749.120,735.530,736.881,743.968,748.005,736.831,737.614,734.464,658.572,655.879,753.828,750.413,627.670,735.928,740.101,742.910,749.417,739.224,738.935,755.543,751.629,628.596,737.516,741.628,744.251,751.192,740.987,740.133,759.046,753.400,630.584,740.520,744.799,747.104,754.986,744.694,742.192,762.304,753.644,632.620,742.884,747.749,749.877,758.796,748.329,743.292,766.211,746.206,634.831,743.499,750.811,753.339,764.630,753.608,739.701,-0.966,0.259,-0.000,1.000,0.782,0.623,-0.188,0.982
26036,2022-12-20,733.720,5.200,95.000,20,4,1,728.340,721.819,708.643,714.540,718.933,726.036,711.701,714.242,710.410,660.379,660.484,729.652,725.587,659.429,710.593,716.578,720.880,727.513,713.380,717.280,731.096,727.015,657.850,712.086,717.969,722.173,728.843,714.917,718.539,734.513,730.104,654.997,715.512,721.232,725.197,732.085,718.555,721.350,738.557,733.089,652.556,719.347,725.063,728.767,736.091,722.913,724.286,747.295,735.243,647.965,726.379,733.076,736.530,745.495,732.814,727.920,-0.866,0.500,-0.000,1.000,0.782,0.623,-0.188,0.982
26037,2022-12-20,705.500,5.100,94.000,20,4,1,698.628,703.043,685.093,692.506,694.390,689.713,685.741,685.650,684.620,656.692,657.377,700.742,704.357,768.081,686.299,691.690,694.546,693.643,686.514,687.830,702.408,705.617,762.205,687.727,693.139,696.061,695.558,688.082,689.506,706.279,708.613,750.039,691.118,696.550,699.567,699.905,691.791,693.294,711.021,712.195,737.427,695.318,700.785,703.842,705.129,696.444,697.682,723.258,719.241,710.883,705.700,711.728,714.845,718.678,708.957,707.100,-0.707,0.707,-0.000,1.000,0.782,0.623,-0.188,0.982
26038,2022-12-20,674.120,5.200,95.000,20,4,1,672.191,678.417,654.461,660.619,662.307,669.278,652.628,655.602,668.184,657.424,659.423,673.589,681.104,775.357,657.496,662.306,667.484,670.080,652.510,657.002,675.185,682.454,774.386,659.155,663.998,669.060,671.512,654.456,658.793,678.984,685.627,770.600,663.008,667.918,672.761,675.053,658.934,662.963,683.818,689.574,764.246,667.781,672.767,677.395,679.727,664.437,668.070,697.709,699.561,743.311,680.840,686.244,690.452,693.759,679.839,681.240,-0.500,0.866,-0.000,1.000,0.782,0.623,-0.188,0.982
26039,2022-12-20,628.540,4.900,94.000,20,4,1,629.573,630.515,629.316,618.485,624.832,623.155,605.674,611.786,628

In [ ]:
!pip install deap
!pip install bitstring

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 9.4 MB/s eta 0:00:00


In [ ]:
!pip install keras

In [ ]:
from sklearn.metrics import mean_squared_error

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
import math
import matplotlib.pyplot as plt
import datetime
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

In [ ]:
df['Tarih'] = pd.to_datetime(df['Tarih'])
df.set_index('Tarih', inplace=True)

In [ ]:
y = df['Dağıtılan'].values
X = df.drop('Dağıtılan', axis=1).values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
col_to_normalize = ['Sıcaklık',
                    'Nem',
                    'Dağıtılan_lag24',
                    'Dağıtılan_lag48',
                    'Dağıtılan_lag96',
                    'Dağıtılan_lag120',
                    'Dağıtılan_lag144',
                    'Dağıtılan_lag168',
                    'Dağıtılan_lag192',
                    'Dağıtılan_lag216',
                    'Dağıtılan_lag240',
                    'dağıtılan8760',
                    'dağıtılan13140',
                    'dağıtılan_ewm_alpha_095_lag_24',
                    'dağıtılan_ewm_alpha_095_lag_48',
                    'dağıtılan_ewm_alpha_095_lag_52',
                    'dağıtılan_ewm_alpha_095_lag_96',
                    'dağıtılan_ewm_alpha_095_lag_120',
                    'dağıtılan_ewm_alpha_095_lag_144',
                  	'dağıtılan_ewm_alpha_095_lag_168',
                    'dağıtılan_ewm_alpha_095_lag_192',
                    'dağıtılan_ewm_alpha_095_lag_216',
                    'dağıtılan_ewm_alpha_09_lag_24',
                    'dağıtılan_ewm_alpha_09_lag_48',
                    'dağıtılan_ewm_alpha_09_lag_52',
                    'dağıtılan_ewm_alpha_09_lag_96',
                    'dağıtılan_ewm_alpha_09_lag_120',
                    'dağıtılan_ewm_alpha_09_lag_144',
                    'dağıtılan_ewm_alpha_09_lag_168',
                    'dağıtılan_ewm_alpha_09_lag_192',
                    'dağıtılan_ewm_alpha_09_lag_216',
                    'dağıtılan_ewm_alpha_08_lag_24',
                    'dağıtılan_ewm_alpha_08_lag_48',
                    'dağıtılan_ewm_alpha_08_lag_52',
                    'dağıtılan_ewm_alpha_08_lag_96',
                    'dağıtılan_ewm_alpha_08_lag_120',
                    'dağıtılan_ewm_alpha_08_lag_144',
                    'dağıtılan_ewm_alpha_08_lag_168',
                    'dağıtılan_ewm_alpha_08_lag_192',
                    'dağıtılan_ewm_alpha_08_lag_216',
                    'dağıtılan_ewm_alpha_07_lag_24',
                    'dağıtılan_ewm_alpha_07_lag_48',
                    'dağıtılan_ewm_alpha_07_lag_52',
                    'dağıtılan_ewm_alpha_07_lag_96',
                    'dağıtılan_ewm_alpha_07_lag_120',
                    'dağıtılan_ewm_alpha_07_lag_144',
                    'dağıtılan_ewm_alpha_07_lag_168',
                    'dağıtılan_ewm_alpha_07_lag_192',
                    'dağıtılan_ewm_alpha_07_lag_216',
                    'dağıtılan_ewm_alpha_05_lag_24',
                    'dağıtılan_ewm_alpha_05_lag_48',
                    'dağıtılan_ewm_alpha_05_lag_52',
                    'dağıtılan_ewm_alpha_05_lag_96',
                    'dağıtılan_ewm_alpha_05_lag_120',
                    'dağıtılan_ewm_alpha_05_lag_144',
                    'dağıtılan_ewm_alpha_05_lag_168',
                    'dağıtılan_ewm_alpha_05_lag_192',
                    'dağıtılan_ewm_alpha_05_lag_216']
X_scaled = scaler.fit_transform(X[col_to_normalize])
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3)

X_train_modified = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_modified = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
def train_evaluate(ga_individual_solution):

  num_neurons1_bits = BitArray(ga_individual_solution[0:9])
  num_neurons2_bits = BitArray(ga_individual_solution[9:18])
  epoch_bits = BitArray(ga_individual_solution[18:25])
  batch_size_bits = BitArray(ga_individual_solution[25:35])
  learning_rate_bits = BitArray(ga_individual_solution[35:])

  num_neurons1 = num_neurons1_bits.uint
  num_neurons2 = num_neurons2_bits.uint
  epoch = epoch_bits.uint
  batch_size = batch_size_bits.uint
  temp = learning_rate_bits.uint
  learning_rate = temp * (math.exp(-9))

  print(f'\nNum of neurons1: {num_neurons1}, Num of neurons2: {num_neurons2}, Epoch: {epoch}, Batch size: {batch_size}, Learning rate: {learning_rate}')

  if num_neurons1 < 10 or num_neurons2 < 10 or epoch < 10 or batch_size  < 32 or learning_rate <= 0:
    return 0,

  model = Sequential()
  model.add(LSTM(num_neurons1, input_shape=(X_train_modified.shape[1], X_train_modified.shape[2]), return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(num_neurons2))
  model.add(Dropout(0.2))
  model.add(Dense(1))

  model.compile(loss='mean_squared_error', optimizer='adam')

  history = model.fit(X_train_modified, y_train, epochs=epoch, batch_size=batch_size, validation_data=(X_test_modified, y_test), verbose=0)

  plt.plot(history.history['loss'], label='Train Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(loc='upper right')
  plt.show()

  y_pred = model.predict(X_test_modified)

  rmse = np.sqrt(np.mean((y_pred - y_test)** 2))
  print(f'Validation RMSE: {rmse}')

  best_solution = ga.run()
  best_batch_size = BitArray(best_solution[25:35]).uint
  best_epoch = BitArray(best_solution[18:25]).uint

  print(f"Optimum Batch Size: {best_batch_size}, Optimum Epoch: {best_epoch}")

In [ ]:
population_size = 100
num_generations = 50
gene_length = 50

In [ ]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n=gene_length)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb=0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

In [ ]:
population = toolbox.population(n=population_size)

In [ ]:
start_time = datetime.now()
print('Start time', start_time)

In [ ]:
r = algorithms.eaSimple(population, toolbox, cxpb=0.4, mutpb=0.1, ngen=num_generations, verbose=False)

In [ ]:
end_time = datetime.now()
print('End time:', end_time)
time_elapsed = end_time - start_time
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

In [ ]:
best_individuals = tools.selBest(population, k=1)
best_num_neurons1 = None
best_num_neurons2 = None
best_epoch = None
best_batch_size = None
best_learning_rate = None

for bi in best_individuals:
  num_neurons1_bits = BitArray(bi[0:9])
  num_neurons2_bits = BitArray(bi[9:18])
  epoch_bits = BitArray(bi[18:25])
  batch_size_bits = BitArray(bi[25:35])
  learning_rate_bits = BitArray(bi[35:])

  best_num_neurons1 = num_neurons1_bits.uint
  best_num_neurons2 = num_neurons2_bits.uint
  best_epoch = epoch_bits.uint
  best_batch_size = batch_size_bits.uint
  temp = learning_rate_bits.uint
  best_learning_rate = temp * (math.exp(-9))
  print(f'\nNum of neurons1: ', best_num_neurons1, '\nNum of neurons2:', best_num_neurons2, '\nEpoch:', best_epoch, '\nBatch_size:', best_batch_size, '\nLearning rate:', best_learning_rate)

NameError: name 'tools' is not defined